# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 50 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.03130


extracting tarball to tmp_2210.03130...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.03132


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.03130/rnaas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2210.03705...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.03721


extracting tarball to tmp_2210.03721...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.03721/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'author-list' from 'tmp_2210.03721/author-list.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'author-list' from 'tmp_2210.03721/Exact window functions/author-list.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03615-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03615) | **The three-dimensional structure of Galactic molecular cloud complexes  out to 2.5 kpc**  |
|| <mark>T. E. Dharmawardena</mark>, et al. -- incl., <mark>C.A.L. Bailer-Jones</mark>, <mark>M. Fouesneau</mark>, <mark>T. Müller</mark>, <mark>J. Henshaw</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| *accepted for publication by MNRAS, 23 pages, 9 figures, 3 tables*|
|**Abstract**| Knowledge of the three-dimensional structure of Galactic molecular clouds is important for understanding how clouds are affected by processes such as turbulence and magnetic fields and how this structure effects star formation within them. Great progress has been made in this field with the arrival of the Gaia mission, which provides accurate distances to $\sim10^{9}$ stars. Combining these distances with extinctions inferred from optical-IR, we recover the three-dimensional structure of 16 Galactic molecular cloud complexes at $\sim1$pc resolution using our novel three-dimensional dust mapping algorithm \texttt{Dustribution}. Using \texttt{astrodendro} we derive a catalogue of physical parameters for each complex. We recover structures with aspect ratios between 1 and 11, i.e.\ everything from near-spherical to very elongated shapes. We find a large variation in cloud environments that is not apparent when studying them in two-dimensions. For example, the nearby California and Orion A clouds look similar on-sky, but we find California to be more sheet-like, and massive, which could explain their different star-formation rates. In Carina, our most distant complex, we observe evidence for dust sputtering, which explains its measured low dust mass. By calculating the total mass of these individual clouds, we demonstrate that it is necessary to define cloud boundaries in three-dimensions in order to obtain an accurate mass; simply integrating the extinction overestimates masses. We find that Larson's relationship on mass vs radius holds true whether you assume a spherical shape for the cloud or take their true extents. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03509-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03509) | **The impact of environment on the lives of disk galaxies as revealed by  SDSS-IV MaNGA**  |
|| Shuang Zhou, et al. -- incl., <mark>Joel R. Brownstein</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| *9 pages, 3 figures, MNRAS accepted*|
|**Abstract**| Environment has long been known to have significant impact on the evolution of galaxies, but here we seek to quantify the subtler differences that might be found in disk galaxies, depending on whether they are isolated, the most massive galaxy in a group (centrals), or a lesser member (satellites). The MaNGA survey allows us to define a large mass-matched sample of 574 galaxies with high-quality integrated spectra in each category. Initial examination of their spectral indices indicates significant differences, particularly in low-mass galaxies. Semi-analytic spectral fitting of a full chemical evolution model to these spectra confirms these differences, with low-mass satellites having a shorter period of star formation and chemical enrichment typical of a closed box, while central galaxies have more extended histories, with evidence of on-going gas accretion over their lifetimes. The derived parameters for gas infall timescale and wind strength suggest that low-mass satellite galaxies have their hot halos of gas effectively removed, while central galaxies retain a larger fraction of gas than isolated galaxies due to the deeper group potential well in which they sit. S0 galaxies form a distinct subset within the sample, particularly at higher masses, but do not bias the inferred lower-mass environmental impact significantly. The consistent picture that emerges underlines the wealth of archaeological information that can be extracted from high-quality spectral data using techniques like semi-analytic spectral fitting. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03130-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03130) | **Mapping Variations of Redshift Distributions with Probability Integral  Transforms**  |
|| J. Myles, et al. -- incl., <mark>I. Harrison</mark>, <mark>J. J. Mohr</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| **|
|**Abstract**| We present a method for mapping variations between probability distribution functions and apply this method within the context of measuring galaxy redshift distributions from imaging survey data. This method, which we name PITPZ for the probability integral transformations it relies on, uses a difference in curves between distribution functions in an ensemble as a transformation to apply to another distribution function, thus transferring the variation in the ensemble to the latter distribution function. This procedure is broadly applicable to the problem of uncertainty propagation. In the context of redshift distributions, for example, the uncertainty contribution due to certain effects can be studied effectively only in simulations, thus necessitating a transfer of variation measured in simulations to the redshift distributions measured from data. We illustrate the use of PITPZ by using the method to propagate photometric calibration uncertainty to redshift distributions of the Dark Energy Survey Year 3 weak lensing source galaxies. For this test case, we find that PITPZ yields a lensing amplitude uncertainty estimate due to photometric calibration error within 1 per cent of the truth, compared to as much as a 30 per cent underestimate when using traditional methods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03645-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03645) | **FAST observations of an extremely active episode of FRB 20201124A: II.  Energy Distribution**  |
|| <mark>Yong-Kun Zhang</mark>, et al. -- incl., <mark>Bing Zhang</mark>, <mark>Di Li</mark>, <mark>Ke-Jia Lee</mark>, <mark>Chun-Feng Zhang</mark>, <mark>Heng Xu</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| *26 pages, 7 figures, accepted for publication in Research in Astronomy and Astrophysics*|
|**Abstract**| We report the properties of more than 800 bursts detected from the repeating fast radio burst (FRB) source FRB 20201124A with the Five-hundred-meter Aperture Spherical radio Telescope (FAST) during an extremely active episode on UTC September 25-28, 2021 in a series of four papers. In this second paper of the series, we mainly focus on the energy distribution of the detected bursts. The event rate initially increased exponentially but the source activity stopped within 24 hours after the 4th day. The detection of 542 bursts in one hour during the fourth day marked the highest event rate detected from one single FRB source so far. The bursts have complex structures in the time-frequency space. We find a double-peak distribution of the waiting time, which can be modeled with two log-normal functions peaking at 51.22 ms and 10.05 s, respectively. Compared with the emission from a previous active episode of the source detected with FAST, the second distribution peak time is smaller, suggesting that this peak is defined by the activity level of the source. We calculate the isotropic energy of the bursts using both a partial bandwidth and a full bandwidth and find that the energy distribution is not significantly changed. We find that an exponentially connected broken-power-law function can fit the cumulative burst energy distribution well, with the lower and higher-energy indices being $-1.22\pm0.01$ and $-4.27\pm0.23$, respectively. Assuming a radio radiative efficiency of $\eta_r = 10^{-4}$, the total isotropic energy of the bursts released during the four days when the source was active is already $3.9\times10^{46}$ erg, exceeding $\sim 23\%$ of the available magnetar dipolar magnetic energy. This challenges the magnetar models invoking an inefficient radio emission (e.g. synchrotron maser models). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03684-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03684) | **Galaxy Zoo: Clump Scout -- Design and first application of a  two-dimensional aggregation tool for citizen science**  |
|| Hugh Dickinson, et al. -- incl., <mark>Chris Lintott</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| *31 pages, 22 figures. Accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| Galaxy Zoo: Clump Scout is a web-based citizen science project designed to identify and spatially locate giant star forming clumps in galaxies that were imaged by the Sloan Digital Sky Survey Legacy Survey. We present a statistically driven software framework that is designed to aggregate two-dimensional annotations of clump locations provided by multiple independent Galaxy Zoo: Clump Scout volunteers and generate a consensus label that identifies the locations of probable clumps within each galaxy. The statistical model our framework is based on allows us to assign false-positive probabilities to each of the clumps we identify, to estimate the skill levels of each of the volunteers who contribute to Galaxy Zoo: Clump Scout and also to quantitatively assess the reliability of the consensus labels that are derived for each subject. We apply our framework to a dataset containing 3,561,454 two-dimensional points, which constitute 1,739,259 annotations of 85,286 distinct subjects provided by 20,999 volunteers. Using this dataset, we identify 128,100 potential clumps distributed among 44,126 galaxies. This dataset can be used to study the prevalence and demographics of giant star forming clumps in low-redshift galaxies. The code for our aggregation software framework is publicly available at: https://github.com/ou-astrophysics/BoxAggregator |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03705-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03705) | **Searching for dilaton fields in the Ly$α$ forest**  |
|| Louis Hamaide, <mark>Hendrik Müller</mark>, David J. E. Marsh |
|*Appeared on*| *2022-10-10*|
|*Comments*| *22 pages, 9 figures, 1 table*|
|**Abstract**| Dilatons (and moduli) couple to the masses and coupling constants of ordinary matter, and these quantities are fixed by the local value of the dilaton field. If, in addition, the dilaton with mass $m_\phi$ contributes to the cosmic dark matter density, then such quantities oscillate in time at the dilaton Compton frequency. We show how these oscillations lead to broadening and shifting of the Voigt profile of the Ly$\alpha$ forest, in a manner that is correlated with the local dark matter density. We further show how tomographic methods allow the effect to be reconstructed by observing the Ly$\alpha$ forest spectrum of distant quasars. We then simulate a large number of quasar lines of sight using the lognormal density field, and forecast the ability of future astronomical surveys to measure this effect. We find that in the ultra low mass range $10^{-32}\text{ eV}\leq m_\phi\leq 10^{-28}\text{ eV}$ upcoming observations can improve over existing limits to the dilaton electron mass and fine structure constant couplings set by fifth force searches by up to five orders of magnitude. Our projected limits apply assuming that the ultralight dilaton makes up a few percent of the dark matter density, consistent with upper limits set by the cosmic microwave background anisotropies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03721) | **What does an interferometer really measure? Including instrument and  data characteristics in the reconstruction of the 21cm power spectrum**  |
|| Adélie Gorce, et al. -- incl., <mark>Adrian Liu</mark>, <mark>Jacqueline N. Hewitt</mark> |
|*Appeared on*| *2022-10-10*|
|*Comments*| *18 pages, 18 figures, submitted to MNRAS. Comments welcome!*|
|**Abstract**| Combining the visibilities measured by an interferometer to form a cosmological power spectrum is a complicated process in which the window functions play a crucial role. In a delay-based analysis, the mapping between instrumental space, made of per-baseline delay spectra, and cosmological space is not a one-to-one relation. Instead, neighbouring modes contribute to the power measured at one point, with their respective contributions encoded in the window functions. To better understand the power spectrum measured by an interferometer, we assess the impact of instrument characteristics and analysis choices on the estimator by deriving its exact window functions, outside of the delay approximation. Focusing on HERA as a case study, we find that observations made with long baselines tend to correspond to enhanced low-k tails of the window functions, which facilitate foreground leakage outside the wedge, whilst the choice of bandwidth and frequency taper can help narrow them down. With the help of simple test cases and more realistic visibility simulations, we show that, apart from tracing mode mixing, the window functions can accurately reconstruct the power spectrum estimator of simulated visibilities. We note that the window functions depend strongly on the chromaticity of the beam, and less on its spatial structure - a Gaussian approximation, ignoring side lobes, is sufficient. Finally, we investigate the potential of asymmetric window functions, down-weighting the contribution of low-k power to avoid foreground leakage. The window functions presented in this work correspond to the latest HERA upper limits for the full Phase I data. They allow an accurate reconstruction of the power spectrum measured by the instrument and can be used in future analyses to confront theoretical models and data directly in cylindrical space. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03215-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03215) | **Characterization of LEO Satellites With All-Sky Photometric Signatures**  |
|| <mark>Harrison Krantz</mark>, Eric C. Pearce, Adam Block |
|*Appeared on*| *2022-10-10*|
|*Comments*| *11 pages, 8 figures, Proceedings of the 2022 AMOS Conference*|
|**Abstract**| We present novel techniques and methodology for unresolved photometric characterization of low-Earth Orbit (LEO) satellites. With the Pomenis LEO Satellite Photometric Survey our team has made over 14,000 observations of Starlink and OneWeb satellites to measure their apparent brightness. From the apparent brightness of each satellite, we calculate a new metric: the effective albedo, which quantifies the specularity of the reflecting satellite. Unlike stellar magnitude units, the effective albedo accounts for apparent range and phase angle and enables direct comparison of different satellites. Mapping the effective albedo from multiple observations across the sky produces an all-sky photometric signature which is distinct for each population of satellites, including the various sub-models of Starlink satellites. Space Situational Awareness (SSA) practitioners can use all-sky photometric signatures to differentiate populations of satellites, compare their reflection characteristics, identify unknown satellites, and find anomalous members. To test the efficacy of all-sky signatures for satellite identification, we applied a machine learning classifier algorithm which correctly identified the majority of satellites based solely on the effective albedo metric and with as few as one observation per individual satellite. Our new method of LEO satellite photometric characterization requires no prior knowledge of the satellite's properties and is readily scalable to large numbers of satellites such as those expected with developing communications mega-constellations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.03615.md
    + _build/html/tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_Vol.png
    + _build/html/tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_SphericalRad.png
    + _build/html/tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_Xextent.png
    + _build/html/tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_FillFac.png
    + _build/html/tmp_2210.03615/./Figures/Dense_LowCbar/Cas_GpyPredicted_DensitySlices_alongDist.png
    + _build/html/tmp_2210.03615/./Figures/ThreeD_renders/mn_22_2305_mj_cassiopeia_sun.png
    + _build/html/tmp_2210.03615/./Figures/ThreeD_renders/mn_22_2305_mj_cassiopeia_top.png
    + _build/html/tmp_2210.03615/./Figures/Ext_Features/California_Ext_Cumulative_WithFeatures.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The three-dimensional structure of Galactic molecular cloud complexes out to 2.5 kpc 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.03615-b31b1b.svg)](https://arxiv.org/abs/2210.03615)<mark>Appeared on: 2022-10-10</mark> - _accepted for publication by MNRAS, 23 pages, 9 figures, 3 tables_

</div>
<div id="authors">

<mark><mark>T. E. Dharmawardena</mark></mark>, et al. -- incl., <mark><mark>C.A.L. Bailer-Jones</mark></mark>, <mark><mark>M. Fouesneau</mark></mark>, <mark><mark>T. Müller</mark></mark>, <mark><mark>J. Henshaw</mark></mark>

</div>
<div id="abstract">

**Abstract:** Knowledge of the three-dimensional structure of Galactic molecular clouds is important for understanding how clouds are affected by processes such as turbulence and magnetic fields and how this structure effects star formation within them.Great progress has been made in this field with the arrival of the Gaia mission, which provides accurate distances to$\sim10^{9}$stars. Combining these distances with extinctions inferred from optical-IR, we recover the three-dimensional structure of 16 Galactic molecular cloud complexes at$\sim1$pc resolution using our novel three-dimensional dust mapping algorithm\texttt{Dustribution}.Using\texttt{astrodendro}we derive a catalogue of physical parameters for each complex.We recover structures with aspect ratios between 1 and 11, i.e. everything from near-spherical to very elongated shapes.We find a large variation in cloud environments that is not apparent when studying them in two-dimensions.For example, the nearby California and Orion A clouds look similar on-sky, but we find California to be more sheet-like, and massive, which could explain their different star-formation rates.In Carina, our most distant complex, we observe evidence for dust sputtering, which explains its measured low dust mass.By calculating the total mass of these individual clouds, we demonstrate that it is necessary to define cloud boundaries in three-dimensions in order to obtain an accurate mass; simply integrating the extinction overestimates masses.We find that Larson's relationship on mass vs radius holds true whether you assume a spherical shape for the cloud or take their true extents.

</div>

<div id="div_fig1">

<img src="tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_Vol.png" alt="Fig18.1" width="25%"/><img src="tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_SphericalRad.png" alt="Fig18.2" width="25%"/><img src="tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_Xextent.png" alt="Fig18.3" width="25%"/><img src="tmp_2210.03615/./Figures/RegionParams/Plot_AllRegions_AllTrunksChildren_Mass_vs_FillFac.png" alt="Fig18.4" width="25%"/>

**Figure 18. -** The total mass vs. (top left) volume of structure within contours laid down by \texttt{astrodendro}; (top right) spherical radius; (bottom left) $x$ extent; and (bottom right) filling factor of all trunks (brown filled circles) and leaves (green empty circles) for all the mapped molecular cloud complexes. In all plots the black dashed line represents the Larson's law where $M \propto R^{q}$ with $q = 1.9$. (*fig:Mass_vs_ParamsComp*)

</div>
<div id="div_fig2">

<img src="tmp_2210.03615/./Figures/Dense_LowCbar/Cas_GpyPredicted_DensitySlices_alongDist.png" alt="Fig21.1" width="33%"/><img src="tmp_2210.03615/./Figures/ThreeD_renders/mn_22_2305_mj_cassiopeia_sun.png" alt="Fig21.2" width="33%"/><img src="tmp_2210.03615/./Figures/ThreeD_renders/mn_22_2305_mj_cassiopeia_top.png" alt="Fig21.3" width="33%"/>

**Figure 21. -**  Predicted 3D density structure of the Cassiopeia molecular cloud complex. Left: Slices along the line-of-sight of the predicted 3D density structure. With the Cyan triangle we have marked the mass weighted centroid of the primary trunk (as given in Table \ref{tab:maintrunk_params} placed on the closest distance slice included in the plot. The Cyan $\times$s mark the mass weighted centroids of the interesting features discussed in Sec.\ref{sec:IndGMCs} and highlighted in Table \ref{tab:leaf_params} placed on the closest distance slice included in the plot. The grey hexagon marks the centre of the Cepheus flare shell; Right top: Video of a volume rendering of the predicted 3D density structure which begins from the view as seen from the Sun. It then rotates anti-clockwise about an axis perpendicular to the initial viewing angle. The semi-transparent iso-surfaces mark three different density levels with orange being the least dense to blue being the most dense as shown by the colour bar.; Right bottom: Still image showing the top down view of the predicted 3D density structure of the molecular cloud region using identical rendering to the preceding video. (*fig:Cas_3Ddens*)

</div>
<div id="div_fig3">

<img src="tmp_2210.03615/./Figures/Ext_Features/California_Ext_Cumulative_WithFeatures.png" alt="Fig1" width="100%"/>

**Figure 1. -** The predicted integrated 2D extinction of the California molecular cloud complex as seen from the Sun (i.e: on the plane of the sky). The California spine and bubble are highlighted (*fig:Calif_2D*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

58  publications in the last 7 days.
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers